<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/geemap/tree/master/examples/geemap-and-earth-engine.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/geemap/blob/master/examples/geemap-and-earth-engine.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://mybinder.org/v2/gh/giswqs/geemap/master?filepath=examples/geemap-and-earth-engine.ipynb"><img width=58px src="https://mybinder.org/static/images/logo_social.png" />Run in binder</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/geemap-and-earth-engine.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

## Install Earth Engine API and geemap
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geemap](https://github.com/giswqs/geemap). The **geemap** Python package builds on the [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) package and implements several methods for displaying Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`.
The following script checks if the geemap package has been installed. If not, it will install geemap, which automatically install its dependencies, including earthengine-api and ipyleaflet.

In [1]:
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap

Import and authenticate Earth Engine

In [2]:
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create an interactive map

In [3]:
Map = geemap.Map(center=(40, -100), zoom=4)
Map

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

## Add Earth Engine Python script

In [4]:
# Add Earth Engine dataset
image = ee.Image('USGS/SRTMGL1_003')

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Print the elevation of Mount Everest.
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = image.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

# Add Earth Eninge layers to Map
Map.addLayer(image, vis_params, 'STRM DEM', True, 0.5)
Map.addLayer(xy, {'color': 'red'}, 'Mount Everest')

Mount Everest elevation (m): 8729


## Change map positions

For example, center the map on an Earth Engine object:

In [5]:
Map.centerObject(ee_object=xy, zoom=13)

Set the map center using coordinates (longitude, latitude)

In [6]:
Map.setCenter(lon=-100, lat=40, zoom=4)

## Extract information from Earth Engine data based on user inputs

In [7]:
import geemap
from ipywidgets import Label
from ipyleaflet import Marker, MarkerCluster

Map = geemap.Map(center=(40, -100), zoom=4)
Map.default_style = {'cursor': 'crosshair'}

# Add Earth Engine dataset
image = ee.Image('USGS/SRTMGL1_003')

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Add Earth Eninge layers to Map
Map.addLayer(image, vis_params, 'STRM DEM', True, 0.5)

latlon_label = Label()
elev_label = Label()
display(latlon_label)
display(elev_label)
coordinates = []
markers = []
marker_cluster = MarkerCluster(name="Marker Cluster")
Map.add_layer(marker_cluster)

def handle_interaction(**kwargs):
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'mousemove':
        latlon_label.value = "Coordinates: {}".format(str(latlon))
    elif kwargs.get('type') == 'click':
        coordinates.append(latlon)
#         Map.add_layer(Marker(location=latlon))
        markers.append(Marker(location=latlon))
        marker_cluster.markers = markers
        xy = ee.Geometry.Point(latlon[::-1])
        elev = image.sample(xy, 30).first().get('elevation').getInfo()
        elev_label.value = "Elevation of {}: {} m".format(latlon, elev)
Map.on_interaction(handle_interaction)

Map

Label(value='')

Label(value='')

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [8]:
Map.last_click

[]

In [9]:
import ee
import geemap
from ipyleaflet import *
from bqplot import pyplot as plt

import numpy as np

Map = geemap.Map()
Map.default_style = {'cursor': 'crosshair'}

# Compute the trend of nighttime lights from DMSP.

# Add a band containing image date as years since 1990.
def createTimeBand(img):
  year = img.date().difference(ee.Date('1991-01-01'), 'year')
  return ee.Image(year).float().addBands(img)


NTL = ee.ImageCollection('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS') \
    .select('stable_lights')

# Fit a linear trend to the nighttime lights collection.
collection = NTL.map(createTimeBand)
fit = collection.reduce(ee.Reducer.linearFit())

image = NTL.toBands()
# print(image.bandNames().getInfo())
# xy = ee.Geometry.Point(38.13, 48.05)
# values = image.sample(xy, 500).first()
# values = values.toDictionary().values().getInfo()
# print(values)

figure = plt.figure(1, title='Nighttime Light Trend', layout={'max_height': '250px', 'max_width': '400px'})
# np.random.seed(0)
# n = 200
# x = np.linspace(0.0, 10.0, n)
# y = np.cumsum(np.random.randn(n))
count = collection.size().getInfo()
start_year = 1992
end_year = 2013
x = range(1, count+1)

# y = values
# plt.plot(x, y)
# plt.show()

coordinates = []
markers = []
marker_cluster = MarkerCluster(name="Marker Cluster")
Map.add_layer(marker_cluster)

def handle_interaction(**kwargs):
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'click':
        coordinates.append(latlon)
        markers.append(Marker(location=latlon))
        marker_cluster.markers = markers
        xy = ee.Geometry.Point(latlon[::-1])
        y = image.sample(xy, 500).first().toDictionary().values().getInfo()
        plt.clear()
        plt.plot(x, y)
#         plt.xticks(range(start_year, end_year, 5))
Map.on_interaction(handle_interaction)


# Display a single image
Map.addLayer(ee.Image(collection.select('stable_lights').first()), {'min': 0, 'max': 63}, 'First image')

# Display trend in red/blue, brightness in green.
Map.setCenter(30, 45, 4)
Map.addLayer(fit,
         {'min': 0, 'max': [0.18, 20, -0.18], 'bands': ['scale', 'offset', 'scale']},
         'stable lights trend')

fig_control = WidgetControl(widget=figure, position='bottomright')

Map.add_control(fig_control)

Map

Map(center=[45, 30], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

In [10]:
import ipywidgets as widgets

widgets.RadioButtons(
    options=['pepperoni', 'pineapple', 'anchovies'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Pizza topping:',
    disabled=False
)

RadioButtons(description='Pizza topping:', options=('pepperoni', 'pineapple', 'anchovies'), value='pepperoni')

In [11]:
from ipyleaflet import Map, basemaps, basemap_to_tiles

m = Map(center=(52.204793, 360.121558), zoom=9)

dark_matter_layer = basemap_to_tiles(basemaps.CartoDB.DarkMatter)
m.add_layer(dark_matter_layer)
m

Map(center=[52.204793, 360.121558], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

In [12]:
nasa_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2018-04-08");
m.add_layer(nasa_layer);